<a href="https://colab.research.google.com/github/talgalper/Honours-2021/blob/main/nextstrain_data_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ncbi-blast+
!apt-get install exonerate
!apt-get install dssp
!pip install biostructmap

In [ ]:
import pandas as pd

from collections import defaultdict

import biostructmap

In [ ]:
ns_data = pd.read_csv('nextstrain_ncov_global_diversity.tsv', sep='\t')

s_gene_data = ns_data[ns_data.gene == 'S']

pos = s_gene_data.position
entropy = s_gene_data.entropy

# Default dict will return zero if not in original entropy data.
pos_to_entropy = defaultdict(int, zip(pos, entropy))

len_of_ref_seq = 1275 # Please change to real length

# Create a list that includes zeros.
list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]

with open('global_diversity_processed.txt', 'w') as f:
    for entropy in list_of_values:
        f.write(f"{entropy:0.3f}\n")

In [ ]:
def process_nextstrain_data(filename, seq_len):
    ns_data = pd.read_csv(filename, sep='\t')

    s_gene_data = ns_data[ns_data.gene == 'S']

    pos = s_gene_data.position
    entropy = s_gene_data.entropy

    # Default dict will return zero if not in original entropy data.
    pos_to_entropy = defaultdict(int, zip(pos, entropy))

    len_of_ref_seq = seq_len # Please change to real length

    # Create a list that includes zeros.
    list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]
    return list_of_values

data = process_nextstrain_data('nextstrain_ncov_global_diversity.tsv')

In [ ]:
# Initialise structure object
structure = biostructmap.Structure('6xm0.pdb', 'SARS-CoV-2_Spike')

data = {('A,B,C',): ['global_diversity_processed.txt']}

# Note: Use method='default' for mapping an ordered list of values (must contain values for each residue in the sequence.)
results = structure.map(data, method='default', ref=None, radius=15)

results.write_data_to_pdb_b_factor(fileobj='test_pdb_data_write.pdb')